In [1]:
#получение обучающей, тестовой и тестовой выборки
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.models import load_model
from keras import backend as K
import joblib

In [2]:
loaded_model_away_poly = joblib.load('model_poly_away_new.joblib')
loaded_model_home_poly = joblib.load('model_poly_home_new.joblib')


In [3]:
scaler_filename_home = "scaler_home_new.save"
scaler_filename_away = "scaler_away_new.save"
scaler_home = joblib.load(scaler_filename_home)
scaler_away = joblib.load(scaler_filename_away)

In [4]:
loaded_model_home_neuron = load_model('home_model_new.h5')
loaded_model_away_neuron = load_model('away_model_new.h5')

In [5]:
match_data = pd.read_csv('match_data.csv',index_col='id')

In [6]:
match_data['points'] = match_data.apply(lambda row: 3 if row['goals'] > match_data[(match_data['fixtureID'] == row['fixtureID']) & (match_data['team_id'] != row['team_id'])]['goals'].max() else (0 if row['goals'] < match_data[(match_data['fixtureID'] == row['fixtureID']) & (match_data['team_id'] != row['team_id'])]['goals'].max() else 1), axis=1)

In [7]:
def create_samples(data):
    x = []
    for _, row in data.iterrows():
        new_line = []
        team_data = match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-10::]
        home_data = team_data[team_data['position'] == 'home']
        away_data = team_data[team_data['position'] == 'away']
        points_last = match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-5::]['points'].sum()
        print(match_data[(match_data['team_id'] == row['team_id']) & (match_data['time'] < row['time'])][-5::])
        new_line.extend([row['team_id'], row['name'] ,row['time'], home_data['goals'].mean(), away_data['goals'].mean(), home_data['shots'].mean(), away_data['shots'].mean(), home_data['xg'].mean(), away_data['xg'].mean(), points_last])

        x.extend([new_line]) 
        #y.extend([[1, row['home_goals']], [2, row['away_goals']]])
    return x

In [8]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

conn.request("GET", f"/fixtures?next=8&league=235", headers=headers)
res = conn.getresponse()
data = res.read()
json_data = json.loads(data)
conn.request("GET", f"/fixtures?next=10&league=39", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_2 = json.loads(data)
conn.request("GET", f"/fixtures?next=9&league=78", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_3 = json.loads(data)
conn.request("GET", f"/fixtures?next=10&league=135", headers=headers)
res = conn.getresponse()
data = res.read()
json_data_4 = json.loads(data)
if 'response' in json_data and json_data['response']:
    json_data = json_data['response']

if 'response' in json_data_2 and json_data_2['response']:
    json_data_2 = json_data_2['response']

if 'response' in json_data_3 and json_data_3['response']:
    json_data_3 = json_data_3['response']

if 'response' in json_data_4 and json_data_4['response']:
    json_data_4 = json_data_4['response']

json_data = json_data + json_data_2 + json_data_3 + json_data_4


In [9]:
home_teams = []
away_teams = []
for fixture in json_data:
    home_teams.append([fixture['teams']['home']['id'], fixture['fixture']['timestamp'], fixture['teams']['home']['name']])
    away_teams.append([fixture['teams']['away']['id'], fixture['fixture']['timestamp'], fixture['teams']['away']['name']])
home_teams = pd.DataFrame(home_teams, columns=['team_id', 'time', 'name'])
away_teams = pd.DataFrame(away_teams, columns=['team_id', 'time', 'name'])
home_teams

,team_id,time,name
0,1993,1701514800,Fakel Voronezh
1,2012,1701523800,PFC Sochi
2,2011,1701534600,Nizhny Novgorod
3,555,1701601200,CSKA Moscow
4,621,1701610200,Krasnodar
5,597,1701610200,Lokomotiv Moscow
6,1085,1701619200,Akhmat Grozny
7,1088,1701707400,Dinamo Moscow
8,42,1701529200,Arsenal
9,55,1701529200,Brentford


In [10]:
away_teams

,team_id,time,name
0,2006,1701514800,Baltika
1,1079,1701523800,Krylya Sovetov
2,1084,1701534600,Ural
3,779,1701601200,FC Rostov
4,1080,1701610200,Orenburg
5,596,1701610200,Zenit Saint Petersburg
6,558,1701619200,Spartak Moscow
7,1083,1701707400,Rubin
8,39,1701529200,Wolves
9,1359,1701529200,Luton


In [11]:
#формирование x
x_home = create_samples(home_teams)
x_home = pd.DataFrame(x_home, columns=['team_id','name', 'time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away', 'points'])
x_away = create_samples(away_teams)
x_away = pd.DataFrame(x_away, columns=['team_id','name','time', 'mean_goals_home', 'mean_goals_away', 'shots_home', 'shots_away', 'xg_home', 'xg_away', 'points'])
x_home

      fixtureID  team_id position  goals  shots   xg        time  points
id                                                                      
4314    1042117     1993     home    2.0    3.0  2.0  1698490800       3
4395    1060682     1993     away    0.0    5.0  0.0  1698852600       0
4425    1042121     1993     away    2.0    3.0  2.0  1699104600       3
4498    1042133     1993     home    1.0    6.0  1.0  1699700400       1
4615    1042138     1993     away    0.0    0.0  0.0  1701005400       0
      fixtureID  team_id position  goals  shots    xg        time  points
id                                                                       
4349    1042116     2012     away    0.0    5.0  2.57  1698577200       0
4394    1060682     2012     home    2.0    5.0  2.00  1698852600       3
4484    1042122     2012     home    3.0   10.0  3.00  1699288200       1
4552    1042131     2012     home    0.0    4.0  0.00  1699804800       0
4571    1042134     2012     away    0.0    1

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points
0,1993,Fakel Voronezh,1701514800,1.250000,0.666667,4.000000,3.166667,1.250000,0.666667,7
1,2012,PFC Sochi,1701523800,1.250000,1.000000,6.000000,3.333333,1.250000,1.428333,4
2,2011,Nizhny Novgorod,1701534600,1.500000,0.500000,4.250000,1.500000,1.307500,0.500000,4
3,555,CSKA Moscow,1701601200,2.000000,1.400000,5.200000,3.000000,2.000000,1.400000,8
4,621,Krasnodar,1701610200,2.500000,1.500000,5.500000,5.000000,2.565000,1.500000,7
5,597,Lokomotiv Moscow,1701610200,1.666667,1.750000,4.166667,5.000000,1.811667,1.830000,9
6,1085,Akhmat Grozny,1701619200,1.250000,0.666667,4.750000,3.000000,1.250000,0.666667,1
7,1088,Dinamo Moscow,1701707400,1.666667,1.750000,5.833333,5.750000,1.666667,1.750000,13
8,42,Arsenal,1701529200,2.750000,1.500000,5.500000,3.666667,1.785000,1.440000,9
9,55,Brentford,1701529200,1.400000,0.800000,4.600000,3.600000,1.418000,1.390000,9


In [12]:
x_away

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points
0,2006,Baltika,1701514800,1.500000,0.250000,4.000000,1.750000,1.851667,0.250000,5
1,1079,Krylya Sovetov,1701523800,2.600000,0.800000,7.000000,1.800000,2.600000,0.774000,5
2,1084,Ural,1701534600,1.200000,1.000000,4.800000,3.000000,1.178000,1.000000,3
3,779,FC Rostov,1701601200,1.666667,1.250000,4.500000,4.250000,1.666667,1.250000,8
4,1080,Orenburg,1701610200,1.400000,0.600000,5.600000,4.000000,1.400000,0.600000,5
5,596,Zenit Saint Petersburg,1701610200,2.000000,1.250000,4.666667,5.250000,2.000000,1.250000,10
6,558,Spartak Moscow,1701619200,2.000000,0.500000,6.500000,2.500000,2.000000,0.361667,7
7,1083,Rubin,1701707400,0.600000,1.000000,3.600000,3.400000,0.600000,1.000000,12
8,39,Wolves,1701529200,1.600000,1.600000,3.200000,4.600000,1.164000,1.360000,7
9,1359,Luton,1701529200,1.000000,1.000000,3.400000,2.600000,1.204000,0.918000,5


In [13]:
elo_england_22 = pd.read_csv('./elo/csv/England/2022/ratingElo2022_2023PL.csv')
elo_england_23 = pd.read_csv('./elo/csv/England/2023/ratingElo2023_2024PL.csv')

elo_england_22.set_index('id', inplace=True)
elo_england_23.set_index('id', inplace=True)

elo_england_23['prev_elos'] = elo_england_23.apply(lambda row: elo_england_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_england_22.index else row['prev_elos'], axis=1)
elo_england_23.reset_index(inplace=True)

elo_russian_22 = pd.read_csv('./elo/csv/Russia/2022/ratingElo2022_2023RFPL.csv')
elo_russian_23 = pd.read_csv('./elo/csv/Russia/2023/ratingElo2023_2024RFPL.csv')

elo_russian_22.set_index('id', inplace=True)
elo_russian_23.set_index('id', inplace=True)

elo_russian_23['prev_elos'] = elo_russian_23.apply(lambda row: elo_russian_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_russian_22.index else row['prev_elos'], axis=1)
elo_russian_23.reset_index(inplace=True)

elo_bundes_22 = pd.read_csv('./elo/csv/Germany/2022/ratingElo2022_2023Bundes.csv')
elo_bundes_23 = pd.read_csv('./elo/csv/Germany/2023/ratingElo2023_2024Bundes.csv')

elo_bundes_22.set_index('id', inplace=True)
elo_bundes_23.set_index('id', inplace=True)

elo_bundes_23['prev_elos'] = elo_bundes_23.apply(lambda row: elo_bundes_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_bundes_22.index else row['prev_elos'], axis=1)
elo_bundes_23.reset_index(inplace=True)

elo_italy_22 = pd.read_csv('./elo/csv/Italy/2022/ratingElo2022_2023Italy.csv')
elo_italy_23 = pd.read_csv('./elo/csv/Italy/2023/ratingElo2023_2024Italy.csv')

elo_italy_22.set_index('id', inplace=True)
elo_italy_23.set_index('id', inplace=True)

elo_italy_23['prev_elos'] = elo_italy_23.apply(lambda row: elo_italy_22.loc[row.name]['prev_elos'] + row['prev_elos'] if row.name in elo_italy_22.index else row['prev_elos'], axis=1)
elo_italy_23.reset_index(inplace=True)

all_elo = pd.concat([elo_england_23, elo_russian_23, elo_bundes_23, elo_italy_23])
all_elo.reset_index(inplace=True)

In [14]:
import re, json
def process_row(row):
    return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])
def add_column_elo(frame):
    frame['elo'] = frame.apply(process_row, axis=1)

In [15]:
samples = [x_home, x_away]
for item in samples:
    add_column_elo(item)

C:\Users\arbol\AppData\Local\Temp\ipykernel_19268\1540769638.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])
C:\Users\arbol\AppData\Local\Temp\ipykernel_19268\1540769638.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  return int(all_elo[all_elo['id'] == row['team_id']]['elo_rating'])


In [16]:
x_home['diff_elo'] = x_home['elo'] - x_away['elo']
x_away['diff_elo'] = x_away['elo'] - x_home['elo']

In [17]:
x_home

,team_id,name,time,mean_goals_home,mean_goals_away,shots_home,shots_away,xg_home,xg_away,points,elo,diff_elo
0,1993,Fakel Voronezh,1701514800,1.250000,0.666667,4.000000,3.166667,1.250000,0.666667,7,1578,38
1,2012,PFC Sochi,1701523800,1.250000,1.000000,6.000000,3.333333,1.250000,1.428333,4,1486,-100
2,2011,Nizhny Novgorod,1701534600,1.500000,0.500000,4.250000,1.500000,1.307500,0.500000,4,1514,-29
3,555,CSKA Moscow,1701601200,2.000000,1.400000,5.200000,3.000000,2.000000,1.400000,8,1717,99
4,621,Krasnodar,1701610200,2.500000,1.500000,5.500000,5.000000,2.565000,1.500000,7,1747,165
5,597,Lokomotiv Moscow,1701610200,1.666667,1.750000,4.166667,5.000000,1.811667,1.830000,9,1706,-104
6,1085,Akhmat Grozny,1701619200,1.250000,0.666667,4.750000,3.000000,1.250000,0.666667,1,1541,-117
7,1088,Dinamo Moscow,1701707400,1.666667,1.750000,5.833333,5.750000,1.666667,1.750000,13,1737,117
8,42,Arsenal,1701529200,2.750000,1.500000,5.500000,3.666667,1.785000,1.440000,9,2042,247
9,55,Brentford,1701529200,1.400000,0.800000,4.600000,3.600000,1.418000,1.390000,9,1918,184


In [18]:
x_home_predict = x_home.drop(['time', 'elo', 'name'], axis=1)
x_away_predict = x_away.drop(['time', 'elo', 'name'], axis=1)

In [19]:
x_home_predict = scaler_home.transform(x_home_predict)
x_away_predict = scaler_away.transform(x_away_predict)

In [20]:
poly_home_value = loaded_model_home_poly.predict(x_home_predict)
poly_away_value = loaded_model_away_poly.predict(x_away_predict)

In [21]:
print("Полином:")
for index, row in x_home.iterrows():
    print(f"{row['name']} - {x_away.iloc[index]['name']}: {round(poly_home_value[index], 3)}:{round(poly_away_value[index], 3)}")

Полином:
Fakel Voronezh - Baltika: 0.984:1.061
PFC Sochi - Krylya Sovetov: 0.97:1.274
Nizhny Novgorod - Ural: 0.915:1.09
CSKA Moscow - FC Rostov: 1.237:1.101
Krasnodar - Orenburg: 1.476:1.006
Lokomotiv Moscow - Zenit Saint Petersburg: 1.186:1.442
Akhmat Grozny - Spartak Moscow: 0.902:1.376
Dinamo Moscow - Rubin: 1.321:0.817
Arsenal - Wolves: 1.538:0.996
Brentford - Luton: 1.539:0.797
Burnley - Sheffield Utd: 0.995:1.04
Nottingham Forest - Everton: 1.172:1.239
Newcastle - Manchester United: 1.509:1.071
Bournemouth - Aston Villa: 1.079:1.945
Chelsea - Brighton: 1.373:1.155
Liverpool - Fulham: 1.876:1.119
West Ham - Crystal Palace: 1.15:1.042
Manchester City - Tottenham: 1.853:1.003
SV Darmstadt 98 - FC Koln: 0.876:1.436
Bayern Munich - Union Berlin: 2.303:0.96
RB Leipzig - FC Heidenheim: 1.77:0.786
Borussia Monchengladbach - 1899 Hoffenheim: 1.331:1.065
VfL BOCHUM - VfL Wolfsburg: 1.047:1.411
VfB Stuttgart - Werder Bremen: 1.86:1.031
FSV Mainz 05 - SC Freiburg: 0.872:1.35
Bayer Leverkuse

In [22]:
nueron_home_value = loaded_model_home_neuron.predict(x_home_predict)
neuron_away_value = loaded_model_away_neuron.predict(x_away_predict)

2/2 [==============================] - 0s 0s/step


In [23]:
print("Нейронная сеть:")
for index, row in x_home.iterrows():
    home_goals = round(float(nueron_home_value[index][0]),3)
    away_goals = round(float(neuron_away_value[index][0]), 3)
    print(f"{row['name']} - {x_away.iloc[index]['name']}: {home_goals}:{away_goals}")

Нейронная сеть:
Fakel Voronezh - Baltika: 1.317:0.913
PFC Sochi - Krylya Sovetov: 1.372:1.29
Nizhny Novgorod - Ural: 1.199:1.137
CSKA Moscow - FC Rostov: 1.734:1.143
Krasnodar - Orenburg: 1.99:1.038
Lokomotiv Moscow - Zenit Saint Petersburg: 1.504:1.365
Akhmat Grozny - Spartak Moscow: 1.315:1.24
Dinamo Moscow - Rubin: 1.833:0.888
Arsenal - Wolves: 2.062:1.044
Brentford - Luton: 1.779:0.874
Burnley - Sheffield Utd: 1.499:1.067
Nottingham Forest - Everton: 1.493:1.283
Newcastle - Manchester United: 2.051:1.166
Bournemouth - Aston Villa: 1.37:1.581
Chelsea - Brighton: 1.796:1.254
Liverpool - Fulham: 2.277:1.105
West Ham - Crystal Palace: 1.681:1.091
Manchester City - Tottenham: 2.242:1.204
SV Darmstadt 98 - FC Koln: 1.513:1.306
Bayern Munich - Union Berlin: 2.894:0.915
RB Leipzig - FC Heidenheim: 2.225:0.897
Borussia Monchengladbach - 1899 Hoffenheim: 1.862:1.23
VfL BOCHUM - VfL Wolfsburg: 1.497:1.311
VfB Stuttgart - Werder Bremen: 2.372:1.078
FSV Mainz 05 - SC Freiburg: 1.353:1.241
Bayer